**BASE DE DADOS DNIT**
Nesta eu encontrei dados sobre estradas não pavimentadas a nível nacional ('Federal' ' ' 'Estadual' 'Municipal' 'Distrital')

In [15]:
!pip install geopandas owslib shapely pyproj --quiet

import unicodedata
from docx import Document
from docx.shared import Inches
import geopandas as gpd
from google.colab import files
import numpy as np
from owslib.wfs import WebFeatureService
import pandas as pd
import requests
from shapely.geometry import LineString

In [6]:
!pip install python-docx --quiet
!pip install owslib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.5/240.5 kB 7.1 MB/s eta 0:00:00


In [16]:
# URL base do OWS do DNIT na INDE
ows_url = "https://geoservicos.inde.gov.br/geoserver/DNIT/ows?"

# Cria objeto WFS (padrão 1.1.0 costuma funcionar bem)
wfs = WebFeatureService(url=ows_url, version='1.1.0')

print("Camadas disponíveis no WFS (layer names):\n")
for name, layer in wfs.contents.items():
    print(name, "->", layer.title)

Camadas disponíveis no WFS (layer names):

DNIT:cide_2024_25 -> Base Rodoviária Estadual Unificada, oriunda dos Sistemas Rodoviários Estaduais (SRE) - 2024
DNIT:vmda_2018 -> Modelagem do Volume Médio Diário Anual de Rodovias Federais e Trechos Estaduais - 2018
DNIT:vmda_2019 -> Modelagem do Volume Médio Diário Anual de Rodovias Federais e Trechos Estaduais - 2019
DNIT:vmda_2020 -> Modelagem do Volume Médio Diário Anual de Rodovias Federais e Trechos Estaduais - 2020
DNIT:vmda_2021 -> Modelagem do Volume Médio Diário Anual de Rodovias Federais e Trechos Estaduais - 2021
DNIT:VMDa_Modelagem_2022 -> Modelagem do Volume Médio Diário Anual de Rodovias Federais e Trechos Estaduais - 2022
DNIT:snv_202507a -> Sistema Nacional de Viação (SNV) – 07/2025


In [17]:
layer_name = "DNIT:vmda_2021"  # <-- AJUSTAR para o nome real que aparecer

In [18]:
# Monta URL de GetFeature em GeoJSON
params = dict(
    service='WFS',
    version='1.1.0',
    request='GetFeature',
    typename=layer_name,
    outputFormat='application/json'
)

wfs_url_geojson = ows_url  # mesmo endpoint OWS

response = requests.get(wfs_url_geojson, params=params)

print("Status:", response.status_code)
response.raise_for_status()  # dispara erro se vier 4xx/5xx

# Salva temporariamente em um arquivo GeoJSON
with open("rodovias_dnit.geojson", "wb") as f:
    f.write(response.content)

# Lê com GeoPandas
gdf = gpd.read_file("rodovias_dnit.geojson")

Status: 200


,id,ogc_fid,id_pnct,codigo_br,regiao,unidade_fe,codigo_snv,extensao,superficie,coincidenc,...,relevo_pre,velocidade,classifica,vmda_ab,vmda_ba,vmda_cresc,vmda_decre,vmda_total,legenda,geometry
0,vmda_2021.1,1,29653885.0,403,Nordeste,CE,403BCE0010,2.61,PAV,-,...,Plano,80,None,1628.0,1605.0,1605.0,1628.0,3233.0,2501-5000,"MULTILINESTRING ((-40.11845 -2.90742, -40.1184..."
1,vmda_2021.2,2,29648488.0,-,Sul,PR,-,0.63,-,-,...,-,999,Conector,NaN,NaN,NaN,NaN,NaN,Não Simulável,"MULTILINESTRING ((-52.78756 -24.24674, -52.787..."
2,vmda_2021.3,3,14242.0,102,Nordeste,SE,-,15.15,LEN,-,...,Ondulado,25,Não Pavimentada,NaN,NaN,NaN,NaN,NaN,Não Simulável,"MULTILINESTRING ((-37.04946 -10.50315, -37.065..."
3,vmda_2021.4,4,29647797.0,-,Sul,RS,-,0.16,-,-,...,-,999,Conector,NaN,NaN,NaN,NaN,NaN,Não Simulável,"MULTILINESTRING ((-52.37409 -30.19152, -52.373..."
4,vmda_2021.5,5,7932.0,060,Centro-Oeste,MS,060BMS0430,25.25,PAV,-,...,Ondulado,70,None,2510.0,2583.0,2510.0,2583.0,5093.0,5001-7500,"MULTILINESTRING ((-54.35028 -19.67959, -54.350..."


In [95]:
# Ver CRS original
print("CRS original:", gdf.crs)

# Se estiver em SIRGAS 2000 geográfico (EPSG:4674) ou similar, reprojetamos:
gdf_proj = gdf.to_crs(epsg=5880)

# Calcula comprimento em km
gdf_proj["len_km"] = gdf_proj.geometry.length / 1000.0

gdf_proj[["len_km"]].describe()

CRS original: EPSG:4674


,len_km
count,23100.000000
mean,14.095318
std,41.124020
min,0.010766
25%,2.111487
50%,8.255235
75%,18.263312
max,2430.265293


In [96]:
# Create a new DataFrame with the requested columns
df_export = gdf_proj[['unidade_fe', 'classifica', 'len_km', 'jurisdicao']]

# Save the DataFrame to a CSV file
df_export.to_csv('dados_export.csv', index=False)

print("DataFrame 'dados_export.csv' created and ready for download.")

# Provide a download link
files.download('dados_export.csv')

DataFrame 'dados_export.csv' created and ready for download.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [97]:
print("Colunas disponíveis:\n", gdf.columns)

# Olhar alguns valores das colunas que parecem relevantes
for col in gdf.columns:
    print(f"\nColuna: {col}")
    print(gdf[col].dropna().unique()[:10])

Colunas disponíveis:
 Index(['id', 'ogc_fid', 'id_pnct', 'codigo_br', 'regiao', 'unidade_fe',
       'codigo_snv', 'extensao', 'superficie', 'coincidenc', 'jurisdicao',
       'tipo_link', 'relevo_pre', 'velocidade', 'classifica', 'vmda_ab',
       'vmda_ba', 'vmda_cresc', 'vmda_decre', 'vmda_total', 'legenda',
       'geometry'],
      dtype='object')

Coluna: id
['vmda_2021.1' 'vmda_2021.2' 'vmda_2021.3' 'vmda_2021.4' 'vmda_2021.5'
 'vmda_2021.6' 'vmda_2021.7' 'vmda_2021.8' 'vmda_2021.9' 'vmda_2021.10']

Coluna: ogc_fid
[ 1  2  3  4  5  6  7  8  9 10]

Coluna: id_pnct
[2.9653885e+07 2.9648488e+07 1.4242000e+04 2.9647797e+07 7.9320000e+03
 2.9648775e+07 2.9652169e+07 1.0617000e+04 2.9651789e+07 2.9651796e+07]

Coluna: codigo_br
['403' '-' '102' '060' '634' '170' '408' '130' '357' '999']

Coluna: regiao
['Nordeste' 'Sul' 'Centro-Oeste' 'Sudeste' 'Norte' 'Exterior']

Coluna: unidade_fe
['CE' 'PR' 'SE' 'RS' 'MS' 'SP' 'MG' 'BA' 'PE' 'ES']

Coluna: codigo_snv
['403BCE0010' '-' '060BMS0430'

In [98]:
# Ajuste esses nomes depois de inspecionar as colunas
UF_COLUMN_NAME = "unidade_fe" # The actual column name for the state/UF
CLASSIFICA_COLUMN_NAME = "classifica" # The actual column name for classification

In [99]:
# Garante que colunas existem
for col in [UF_COLUMN_NAME, CLASSIFICA_COLUMN_NAME]:
    if col not in gdf_proj.columns:
        raise ValueError(f"Coluna {col} não encontrada, ajuste UF_COLUMN_NAME / CLASSIFICA_COLUMN_NAME.")

km_por_uf_pav = (
    gdf_proj
    .groupby([UF_COLUMN_NAME, CLASSIFICA_COLUMN_NAME], dropna=False)["len_km"]
    .sum()
    .reset_index()
    .sort_values([UF_COLUMN_NAME, CLASSIFICA_COLUMN_NAME])
)

,unidade_fe,classifica,len_km
0,AC,Conector,69.764461
1,AC,Não Pavimentada,591.935838
2,AC,Não Simulável,429.169032
3,AC,Referencial,68.160318
4,AC,Travessia - Não Simulável,0.243987
5,AC,NaN,994.404353
6,AL,Conector,158.849386
7,AL,Não Pavimentada,418.827140
8,AL,Não Simulável,428.039991
9,AL,Referencial,181.698404


In [100]:
km_por_uf_pav.to_csv('km_por_uf_pav.csv', index=False)
print("DataFrame 'km_por_uf_pav' salvo como 'km_por_uf_pav.csv'")

DataFrame 'km_por_uf_pav' salvo como 'km_por_uf_pav.csv'


In [101]:
df = pd.read_csv('/content/dados_export.csv', sep=',')

,unidade_fe,classifica,len_km,jurisdicao
0,CE,NaN,2.686184,FEDERAL
1,PR,Conector,0.633902,-
2,SE,Não Pavimentada,15.555274,ESTADUAL
3,RS,Conector,0.163517,-
4,MS,NaN,25.250371,FEDERAL


In [102]:
missing_values = df.isnull().sum()
print("Número de valores faltantes por coluna:")
print(missing_values)

Número de valores faltantes por coluna:
unidade_fe       0
classifica    9211
len_km           0
jurisdicao       0
dtype: int64


In [103]:
# Assuming UF_COLUMN_NAME and CLASSIFICA_COLUMN_NAME are defined from previous cells
# Regenerate km_por_uf_pav to include 'jurisdicao' in the grouping
km_por_uf_pav = (
    gdf_proj
    .groupby([UF_COLUMN_NAME, CLASSIFICA_COLUMN_NAME, 'jurisdicao'], dropna=False)["len_km"]
    .sum()
    .reset_index()
    .sort_values([UF_COLUMN_NAME, CLASSIFICA_COLUMN_NAME, 'jurisdicao'])
)

print("DataFrame 'km_por_uf_pav' (com jurisdição) gerado:")

DataFrame 'km_por_uf_pav' (com jurisdição) gerado:


,unidade_fe,classifica,jurisdicao,len_km
0,AC,Conector,-,69.764461
1,AC,Não Pavimentada,ESTADUAL,423.460840
2,AC,Não Pavimentada,FEDERAL,168.474998
3,AC,Não Simulável,ESTADUAL,187.719205
4,AC,Não Simulável,FEDERAL,241.449827


In [104]:
sum_len_km_nao_pav_federal = km_por_uf_pav[
    (km_por_uf_pav['classifica'] == 'Não Pavimentada') &
    (km_por_uf_pav['jurisdicao'] == 'Federal')
]['len_km'].sum()

print(f"Soma de len_km para 'Não Pavimentada' e 'Federal': {sum_len_km_nao_pav_federal:.2f} Km")

Soma de len_km para 'Não Pavimentada' e 'Federal': 0.00 Km


**MALHA PAVIMENTADA**

Os seguintes dados são provenientes do SNV (Sistema Nacional de Viação)

In [105]:
df1 = pd.read_csv('/content/pav.csv', sep=',', on_bad_lines='warn')

,BR,UF,Tipo de trecho,Desc Coinc,Código,Local de Início,Local de Fim,km inicial,km final,Extensão,Superfície Federal,Obras,Federal Coincidente,Administração,Ato legal,Estadual Coincidente,Superfície Est. Coincidente,Jurisdição,Superfície,Unidade Local
0,10,DF,Eixo Principal,-,010BDF0015,ENTR DF-440,ACESSO I SOBRADINHO,2.4,6.0,3.6,DUP,NaN,010BDF0015;020BDF0015;030BDF0015,Convênio Adm.Federal/Estadual,NaN,NaN,NaN,Federal,PAV,Brasília
1,10,DF,Eixo Principal,-,010BDF0016,ACESSO I SOBRADINHO,ACESSO II SOBRADINHO,6.0,8.3,2.3,DUP,NaN,010BDF0016;020BDF0016;030BDF0016,Convênio Adm.Federal/Estadual,NaN,NaN,NaN,Federal,PAV,Brasília
2,10,DF,Eixo Principal,-,010BDF0018,ACESSO II SOBRADINHO,ENTR DF-230,8.3,18.2,9.9,DUP,NaN,010BDF0018;020BDF0018;030BDF0018,Convênio Adm.Federal/Estadual,NaN,NaN,NaN,Federal,PAV,Brasília
3,10,DF,Eixo Principal,-,010BDF0020,ENTR DF-230,ENTR DF-128,18.2,22.0,3.8,DUP,NaN,010BDF0020;020BDF0020;030BDF0020,Convênio Adm.Federal/Estadual,NaN,NaN,NaN,Federal,PAV,Brasília
4,10,DF,Eixo Principal,-,010BDF0022,ENTR DF-128,P/PLANALTINA,22.0,25.0,3.0,DUP,NaN,010BDF0022;020BDF0022;030BDF0022,Convênio Adm.Federal/Estadual,NaN,NaN,NaN,Federal,PAV,Brasília


In [106]:
df1.columns = df1.columns.str.lower()
print("Colunas do DataFrame df1 convertidas para minúsculas:")

Colunas do DataFrame df1 convertidas para minúsculas:


,br,uf,tipo de trecho,desc coinc,código,local de início,local de fim,km inicial,km final,extensão,superfície federal,obras,federal coincidente,administração,ato legal,estadual coincidente,superfície est. coincidente,jurisdição,superfície,unidade local
0,10,DF,Eixo Principal,-,010BDF0015,ENTR DF-440,ACESSO I SOBRADINHO,2.4,6.0,3.6,DUP,NaN,010BDF0015;020BDF0015;030BDF0015,Convênio Adm.Federal/Estadual,NaN,NaN,NaN,Federal,PAV,Brasília
1,10,DF,Eixo Principal,-,010BDF0016,ACESSO I SOBRADINHO,ACESSO II SOBRADINHO,6.0,8.3,2.3,DUP,NaN,010BDF0016;020BDF0016;030BDF0016,Convênio Adm.Federal/Estadual,NaN,NaN,NaN,Federal,PAV,Brasília
2,10,DF,Eixo Principal,-,010BDF0018,ACESSO II SOBRADINHO,ENTR DF-230,8.3,18.2,9.9,DUP,NaN,010BDF0018;020BDF0018;030BDF0018,Convênio Adm.Federal/Estadual,NaN,NaN,NaN,Federal,PAV,Brasília
3,10,DF,Eixo Principal,-,010BDF0020,ENTR DF-230,ENTR DF-128,18.2,22.0,3.8,DUP,NaN,010BDF0020;020BDF0020;030BDF0020,Convênio Adm.Federal/Estadual,NaN,NaN,NaN,Federal,PAV,Brasília
4,10,DF,Eixo Principal,-,010BDF0022,ENTR DF-128,P/PLANALTINA,22.0,25.0,3.0,DUP,NaN,010BDF0022;020BDF0022;030BDF0022,Convênio Adm.Federal/Estadual,NaN,NaN,NaN,Federal,PAV,Brasília


In [107]:
def remove_accents_and_replace_spaces(text):
    # Remove accents
    nfkd_form = unicodedata.normalize('NFKD', text)
    to_ascii = nfkd_form.encode('ascii', 'ignore').decode('utf-8')
    # Replace spaces with underscores
    return to_ascii.replace(' ', '_')

df1.columns = [remove_accents_and_replace_spaces(col) for col in df1.columns]

print("Colunas do DataFrame df1 após a limpeza:")

Colunas do DataFrame df1 após a limpeza:


,br,uf_,tipo_de_trecho,desc_coinc,codigo,local_de_inicio,local_de_fim,km_inicial,km_final,extensao,superficie_federal,obras,federal_coincidente,administracao,ato_legal,estadual_coincidente,superficie_est._coincidente,jurisdicao,superficie,unidade_local
0,10,DF,Eixo Principal,-,010BDF0015,ENTR DF-440,ACESSO I SOBRADINHO,2.4,6.0,3.6,DUP,NaN,010BDF0015;020BDF0015;030BDF0015,Convênio Adm.Federal/Estadual,NaN,NaN,NaN,Federal,PAV,Brasília
1,10,DF,Eixo Principal,-,010BDF0016,ACESSO I SOBRADINHO,ACESSO II SOBRADINHO,6.0,8.3,2.3,DUP,NaN,010BDF0016;020BDF0016;030BDF0016,Convênio Adm.Federal/Estadual,NaN,NaN,NaN,Federal,PAV,Brasília
2,10,DF,Eixo Principal,-,010BDF0018,ACESSO II SOBRADINHO,ENTR DF-230,8.3,18.2,9.9,DUP,NaN,010BDF0018;020BDF0018;030BDF0018,Convênio Adm.Federal/Estadual,NaN,NaN,NaN,Federal,PAV,Brasília
3,10,DF,Eixo Principal,-,010BDF0020,ENTR DF-230,ENTR DF-128,18.2,22.0,3.8,DUP,NaN,010BDF0020;020BDF0020;030BDF0020,Convênio Adm.Federal/Estadual,NaN,NaN,NaN,Federal,PAV,Brasília
4,10,DF,Eixo Principal,-,010BDF0022,ENTR DF-128,P/PLANALTINA,22.0,25.0,3.0,DUP,NaN,010BDF0022;020BDF0022;030BDF0022,Convênio Adm.Federal/Estadual,NaN,NaN,NaN,Federal,PAV,Brasília


In [108]:
soma_extensao_pav_federal_novo = df1[
    (df1['superficie'] == 'PAV') &
    (df1['jurisdicao'] == 'Federal')
]['extensao'].sum()

print(f"Soma da extensão para superfície 'PAV' e jurisdição 'Federal': {soma_extensao_pav_federal_novo:.2f} Km")

Soma da extensão para superfície 'PAV' e jurisdição 'Federal': 73724.43 Km


In [109]:
missing_values_df1 = df1.isnull().sum()
print("Número de valores faltantes por coluna em df1:")
print(missing_values_df1)

Número de valores faltantes por coluna em df1:
br                                0
uf_                               0
tipo_de_trecho                    0
desc_coinc                        0
codigo                            0
local_de_inicio                   0
local_de_fim                      0
km_inicial                        0
km_final                          0
extensao                          0
superficie_federal                0
obras                          7414
federal_coincidente               0
administracao                     0
ato_legal                      7570
estadual_coincidente           5984
superficie_est._coincidente    5991
jurisdicao                        0
superficie                        0
unidade_local                  2566
dtype: int64


In [110]:
unique_jurisdicao = df1['superficie'].unique()
print("Valores únicos na coluna 'superficie':")
print(unique_jurisdicao)

Valores únicos na coluna 'superficie':
['PAV' 'PLA' 'N_PAV']


In [111]:
unique_jurisdicao = df1['jurisdicao'].unique()
print("Valores únicos na coluna 'jurisdicao':")
print(unique_jurisdicao)

Valores únicos na coluna 'jurisdicao':
['Federal' 'Distrital' 'Estadual' 'Municipal']


In [112]:
# Filtrar apenas trechos com superfície pavimentada
mask_pav = gdf['superficie'] == 'PAV'

total_extensao_pav = gdf.loc[mask_pav, 'extensao'].sum()

print(f"Extensão total pavimentada (superficie == 'PAV'): {total_extensao_pav:.3f} km")

Extensão total pavimentada (superficie == 'PAV'): 175196.220 km


In [113]:
# Normalizar nomes de colunas
gdf.columns = [c.strip().lower() for c in gdf.columns]

# Garantir tipo numérico
gdf['extensao'] = pd.to_numeric(gdf['extensao'], errors='coerce')

# Normalizar texto das colunas de interesse
gdf['superficie'] = gdf['superficie'].astype(str).str.upper().str.strip()
gdf['jurisdicao'] = gdf['jurisdicao'].astype(str).str.upper().str.strip()

# Máscaras: pavimentada + federal
mask_pav = gdf['superficie'] == 'PAV'
# se na base estiver "FEDERAL", "RODOVIA FEDERAL", etc., o contains ajuda:
mask_fed = gdf['jurisdicao'].str.contains('FEDERAL', na=False)

# Extensão total pavimentada em rodovias federais
total_extensao_pav_federal = gdf.loc[mask_pav & mask_fed, 'extensao'].sum()

print(f"Extensão total pavimentada em rodovias FEDERAIS: {total_extensao_pav_federal:.3f} km")

Extensão total pavimentada em rodovias FEDERAIS: 78047.220 km


In [114]:
# Create a new DataFrame with only the specified columns
df_export = df1[['extensao', 'uf_', 'jurisdicao', 'superficie']]

# Save the DataFrame to a CSV file named 'pav1.csv'
df_export.to_csv('pav1.csv', index=False)

print("DataFrame 'pav1.csv' created with selected columns and ready for download.")

# Provide a download link
files.download('pav1.csv')

DataFrame 'pav1.csv' created with selected columns and ready for download.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**AVALIAÇÃO POR UF**

In [115]:
df = pd.read_csv('/content/pav1.csv', sep=',')

,extensao,uf_,jurisdicao,superficie
0,3.6,DF,Federal,PAV
1,2.3,DF,Federal,PAV
2,9.9,DF,Federal,PAV
3,3.8,DF,Federal,PAV
4,3.0,DF,Federal,PAV


In [116]:
unique_jurisdicao = df['uf_'].unique()
print("Valores únicos na coluna 'uf_':")
print(unique_jurisdicao)

Valores únicos na coluna 'uf_':
['DF' 'GO' 'MA' 'PA' 'TO' 'PI' 'BA' 'CE' 'RJ' 'MG' 'SP' 'MS' 'MT' 'ES'
 'RN' 'SC' 'AL' 'PB' 'PE' 'PR' 'RS' 'SE' 'AP' 'RR' 'AM' 'RO' 'AC']


In [117]:
# 1. Normalizar nomes de colunas (evita problema de maiúscula/minúscula)
df.columns = [c.strip().lower() for c in df.columns]

# 2. Garantir que 'extensao' é numérica
df['extensao'] = pd.to_numeric(df['extensao'], errors='coerce')

# 3. Descobrir a coluna de UF (pega a primeira que contenha 'uf' no nome)
uf_candidates = [c for c in df.columns if 'uf' in c]

if not uf_candidates:
    raise ValueError("Não encontrei nenhuma coluna com 'uf' no nome. Confira os nomes das colunas.")

uf_col = uf_candidates[0]  # ou troque manualmente se preferir, ex.: uf_col = 'uf'

print(f"Usando a coluna de UF: '{uf_col}'")

# 4. Somar extensão por UF
km_por_uf = (
    df
    .groupby(uf_col, dropna=False)['extensao']
    .sum()
    .reset_index()
    .rename(columns={'extensao': 'km_total_pav'})
    .sort_values('km_total_pav', ascending=False)
)

km_por_uf

Usando a coluna de UF: 'uf_'


,uf_,km_total_pav
10,MG,18892.21
4,BA,12680.50
22,RS,9672.50
13,PA,8762.52
12,MT,7428.80
17,PR,7111.10
8,GO,6931.70
25,SP,6824.20
2,AM,6476.10
16,PI,5351.20


In [118]:
# Normalizar nomes de colunas
df.columns = [c.strip().lower() for c in df.columns]

# Garantir que 'extensao' é numérica
df['extensao'] = pd.to_numeric(df['extensao'], errors='coerce')

# Identificar coluna de UF (ou defina manualmente, ex.: uf_col = 'uf')
uf_candidates = [c for c in df.columns if 'uf' in c]
if not uf_candidates:
    raise ValueError("Não encontrei nenhuma coluna com 'uf' no nome. Ajuste manualmente o nome da coluna de UF.")

uf_col = uf_candidates[0]
print(f"Usando coluna de UF: {uf_col}")

# Garantir que 'jurisdicao' existe
if 'jurisdicao' not in df.columns:
    raise ValueError("Coluna 'jurisdicao' não encontrada. Confira o nome exato no DataFrame.")

# Normalizar texto da jurisdição
df['jurisdicao'] = df['jurisdicao'].astype(str).str.upper().str.strip()

# Filtrar apenas rodovias federais
mask_federal = df['jurisdicao'].str.contains('FEDERAL', na=False)

df_federal = df[mask_federal].copy()

# Somar extensão por UF apenas para jurisdição federal
km_federal_por_uf = (
    df_federal
      .groupby(uf_col, dropna=False)['extensao']
      .sum()
      .reset_index()
      .rename(columns={'extensao': 'km_pav_federal'})
      .sort_values('km_pav_federal', ascending=False)
)

km_federal_por_uf

Usando coluna de UF: uf_


,uf_,km_pav_federal
10,MG,11268.41
4,BA,10372.30
13,PA,8521.62
22,RS,7316.20
12,MT,6914.10
2,AM,6375.00
17,PR,5717.30
11,MS,4777.30
8,GO,4655.20
9,MA,4271.40


In [119]:
comparacao_km_uf = pd.merge(km_por_uf, km_federal_por_uf, on='uf_', how='outer').fillna(0)
display(comparacao_km_uf.sort_values('km_total_pav', ascending=False))

,uf_,km_total_pav,km_pav_federal
10,MG,18892.21,11268.41
4,BA,12680.50,10372.30
22,RS,9672.50,7316.20
13,PA,8762.52,8521.62
12,MT,7428.80,6914.10
17,PR,7111.10,5717.30
8,GO,6931.70,4655.20
25,SP,6824.20,2149.30
2,AM,6476.10,6375.00
16,PI,5351.20,4043.20


In [120]:
df_final = comparacao_km_uf.copy()

,uf_,km_total_pav,km_pav_federal
0,AC,1668.03,1567.63
1,AL,1103.60,929.40
2,AM,6476.10,6375.00
3,AP,1299.70,1299.70
4,BA,12680.50,10372.30


In [121]:
df = pd.read_csv('/content/dados_export.csv', sep=',')

,unidade_fe,classifica,len_km,jurisdicao
0,CE,NaN,2.686184,FEDERAL
1,PR,Conector,0.633902,-
2,SE,Não Pavimentada,15.555274,ESTADUAL
3,RS,Conector,0.163517,-
4,MS,NaN,25.250371,FEDERAL


In [122]:
df = df.rename(columns={'unidade_fe': 'uf', 'len_km': 'extensao_npav'})

,uf,classifica,extensao_npav,jurisdicao
0,CE,NaN,2.686184,FEDERAL
1,PR,Conector,0.633902,-
2,SE,Não Pavimentada,15.555274,ESTADUAL
3,RS,Conector,0.163517,-
4,MS,NaN,25.250371,FEDERAL


In [123]:
# 1. Normalizar nomes de colunas (evita problema de maiúscula/minúscula)
df.columns = [c.strip().lower() for c in df.columns]

# 2. Garantir que 'extensao_npav' é numérica
df['extensao_npav'] = pd.to_numeric(df['extensao_npav'], errors='coerce')

# 3. Descobrir a coluna de UF (pega a primeira que contenha 'uf' no nome)
uf_candidates = [c for c in df.columns if 'uf' in c]

if not uf_candidates:
    raise ValueError("Não encontrei nenhuma coluna com 'uf' no nome. Confira os nomes das colunas.")

uf_col = uf_candidates[0]  # ou troque manualmente se preferir, ex.: uf_col = 'uf'

print(f"Usando a coluna de UF: '{uf_col}'")

# 4. Somar extensão por UF
km_por_uf_npav = (
    df
    .groupby(uf_col, dropna=False)['extensao_npav']
    .sum()
    .reset_index()
    .rename(columns={'extensao_npav': 'km_total_npav'})
    .sort_values('km_total_npav', ascending=False)
)

km_por_uf_npav

Usando a coluna de UF: 'uf'


,uf,km_total_npav
11,MG,34000.647406
13,MT,26100.326753
4,BA,24608.794393
8,EX,24314.742805
9,GO,22822.297562
26,SP,21343.951497
23,RS,17230.408104
18,PR,16134.607900
12,MS,14070.275594
27,TO,12668.471172


In [124]:
# Normalizar nomes de colunas
df.columns = [c.strip().lower() for c in df.columns]

# Garantir que 'extensao' é numérica
df['extensao_npav'] = pd.to_numeric(df['extensao_npav'], errors='coerce')

# Identificar coluna de UF (ou defina manualmente, ex.: uf_col = 'uf')
uf_candidates = [c for c in df.columns if 'uf' in c]
if not uf_candidates:
    raise ValueError("Não encontrei nenhuma coluna com 'uf' no nome. Ajuste manualmente o nome da coluna de UF.")

uf_col = uf_candidates[0]
print(f"Usando coluna de UF: {uf_col}")

# Garantir que 'jurisdicao' existe
if 'jurisdicao' not in df.columns:
    raise ValueError("Coluna 'jurisdicao' não encontrada. Confira o nome exato no DataFrame.")

# Normalizar texto da jurisdição
df['jurisdicao'] = df['jurisdicao'].astype(str).str.upper().str.strip()

# Filtrar apenas rodovias federais
mask_federal = df['jurisdicao'].str.contains('FEDERAL', na=False)

df_federal = df[mask_federal].copy()

# Somar extensão por UF apenas para jurisdição federal
km_federal_por_uf = (
    df_federal
      .groupby(uf_col, dropna=False)['extensao_npav']
      .sum()
      .reset_index()
      .rename(columns={'extensao_npav': 'km_pav_federal'})
      .sort_values('km_pav_federal', ascending=False)
)

km_federal_por_uf

# Normalizar nomes de colunas
df.columns = [c.strip().lower() for c in df.columns]

# Garantir que 'extensao' é numérica
df['extensao_npav'] = pd.to_numeric(df['extensao_npav'], errors='coerce')

# Identificar coluna de UF (ou defina manualmente, ex.: uf_col = 'uf')
uf_candidates = [c for c in df.columns if 'uf' in c]
if not uf_candidates:
    raise ValueError("Não encontrei nenhuma coluna com 'uf' no nome. Ajuste manualmente o nome da coluna de UF.")

uf_col = uf_candidates[0]
print(f"Usando coluna de UF: {uf_col}")

# Garantir que 'jurisdicao' existe
if 'jurisdicao' not in df.columns:
    raise ValueError("Coluna 'jurisdicao' não encontrada. Confira o nome exato no DataFrame.")

# Normalizar texto da jurisdição
df['jurisdicao'] = df['jurisdicao'].astype(str).str.upper().str.strip()

# Filtrar apenas rodovias federais
mask_federal = df['jurisdicao'].str.contains('FEDERAL', na=False)

df_federal = df[mask_federal].copy()

# Somar extensão por UF apenas para jurisdição federal
km_federal_por_uf = (
    df_federal
      .groupby(uf_col, dropna=False)['extensao_npav']
      .sum()
      .reset_index()
      .rename(columns={'extensao_npav': 'km_npav_federal'})
      .sort_values('km_npav_federal', ascending=False)
)

km_federal_por_uf

Usando coluna de UF: uf
Usando coluna de UF: uf


,uf,km_npav_federal
10,MG,15720.562674
4,BA,9505.323990
22,RS,7972.161868
13,PA,6032.172112
8,GO,5840.932539
25,SP,5683.510212
12,MT,5252.072973
17,PR,5102.396326
2,AM,4902.247125
11,MS,4241.103111


In [125]:
# garantir nome correto da uf
df_final = df_final.rename(columns={'uf_': 'uf'})

# manter apenas as colunas relevantes de cada DF para evitar duplicação
df_final = df_final[['uf', 'km_total_pav', 'km_pav_federal']]

df_final = (
    df_final
    .merge(km_por_uf_npav[['uf', 'km_total_npav']], on='uf', how='outer')
    .merge(km_federal_por_uf[['uf', 'km_npav_federal']], on='uf', how='outer')
)

display(df_final.sort_values('km_total_pav', ascending=False))

,uf,km_total_pav,km_pav_federal,km_total_npav,km_npav_federal
11,MG,18892.21,11268.41,34000.647406,15720.562674
4,BA,12680.50,10372.30,24608.794393,9505.323990
23,RS,9672.50,7316.20,17230.408104,7972.161868
14,PA,8762.52,8521.62,12128.172786,6032.172112
13,MT,7428.80,6914.10,26100.326753,5252.072973
18,PR,7111.10,5717.30,16134.607900,5102.396326
9,GO,6931.70,4655.20,22822.297562,5840.932539
26,SP,6824.20,2149.30,21343.951497,5683.510212
2,AM,6476.10,6375.00,9600.766443,4902.247125
17,PI,5351.20,4043.20,12043.220434,4027.127403


In [ ]:
df_final.to_csv('extensao_rodovias_por_uf_pav_npav.csv', index=False)

In [132]:
# -------------------------------------------------------------------
# 1) DATAFRAME DE TRABALHO PARA EXPORTAÇÃO
# -------------------------------------------------------------------
df_export = df_final.copy()

# remover linhas das UFs 'EX' e 'AL' (caso existam)
df_export = df_export[~df_export['uf'].astype(str).str.upper().isin(['EX', 'AL'])]

# função de formatação PT-BR
def format_br(value):
    """
    Converte número (int/float ou string numérica) para '1.234,56'.
    Mantém não numéricos como estão.
    """
    if pd.isna(value):
        return ""
    # se vier como string, tentar converter
    if isinstance(value, str):
        v = value.strip()
        try:
            num = float(v.replace('.', '').replace(',', '.'))
        except ValueError:
            return v
        value = num

    return f"{float(value):,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

# colunas numéricas que você quer formatar
cols_num = ['km_total_pav', 'km_pav_federal', 'km_total_npav', 'km_npav_federal']

# Garante que as colunas são numéricas antes de formatar
for col in cols_num:
    df_export[col] = pd.to_numeric(df_export[col], errors='coerce')

# Remove linhas que possam ter NaN após a conversão (se houver dados não numéricos)
df_export.dropna(subset=cols_num, inplace=True)

# Agora aplica a função de formatação às colunas
for col in cols_num:
    df_export[col] = df_export[col].apply(format_br)

In [133]:
# -------------------------------------------------------------------
# 2) GERAR DOCUMENTO WORD
# -------------------------------------------------------------------
document = Document()
document.add_heading('Tabela de Pavimentação e Não Pavimentação por UF', level=1)

# tabela com cabeçalho
table = document.add_table(rows=1, cols=df_export.shape[1])
table.style = 'Table Grid'

# cabeçalho
hdr_cells = table.rows[0].cells
for j, col_name in enumerate(df_export.columns):
    hdr_cells[j].text = str(col_name)

# linhas de dados
for _, row in df_export.iterrows():
    row_cells = table.add_row().cells
    for j, cell_value in enumerate(row):
        row_cells[j].text = str(cell_value)

# salvar e baixar
file_name = 'tabela_pav_npav.docx'
document.save(file_name)

print("Dimensão do dataframe exportado:", df_export.shape)
files.download(file_name)

Dimensão do dataframe exportado: (26, 5)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [134]:
display(df_final)

,uf,km_total_pav,km_pav_federal,km_total_npav,km_npav_federal
0,AC,1668.03,1567.63,2153.677989,1426.938552
1,AL,1103.60,929.40,2925.496970,909.478373
2,AM,6476.10,6375.00,9600.766443,4902.247125
3,AP,1299.70,1299.70,1471.456775,1166.556267
4,BA,12680.50,10372.30,24608.794393,9505.323990
5,CE,3701.59,2927.29,11737.827488,3195.032631
6,DF,517.00,362.00,1475.390028,375.403378
7,ES,1752.00,1377.60,5361.362842,1375.285720
8,EX,NaN,NaN,24314.742805,NaN
9,GO,6931.70,4655.20,22822.297562,5840.932539


In [135]:
display(df_final)

,uf,km_total_pav,km_pav_federal,km_total_npav,km_npav_federal
0,AC,1668.03,1567.63,2153.677989,1426.938552
1,AL,1103.60,929.40,2925.496970,909.478373
2,AM,6476.10,6375.00,9600.766443,4902.247125
3,AP,1299.70,1299.70,1471.456775,1166.556267
4,BA,12680.50,10372.30,24608.794393,9505.323990
5,CE,3701.59,2927.29,11737.827488,3195.032631
6,DF,517.00,362.00,1475.390028,375.403378
7,ES,1752.00,1377.60,5361.362842,1375.285720
8,EX,NaN,NaN,24314.742805,NaN
9,GO,6931.70,4655.20,22822.297562,5840.932539


**Tabela por porte**

In [ ]:
# Definição das classes em quilogramas
dados = [
    {
        "porte": "pequeno",
        "limite_inferior_kg": 0.0,
        "limite_superior_kg": 0.5,
        "definicao": "≤ 0,5 kg"
    },
    {
        "porte": "médio",
        "limite_inferior_kg": 0.5,
        "limite_superior_kg": 3.0,
        "definicao": "0,5 – 3 kg"
    },
    {
        "porte": "grande",
        "limite_inferior_kg": 3.0,
        "limite_superior_kg": 20.0,
        "definicao": "3 – 20 kg"
    },
    {
        "porte": "mega",
        "limite_inferior_kg": 20.0,
        "limite_superior_kg": None,
        "definicao": "> 20 kg"
    }
]

df_classes = pd.DataFrame(dados)
df_classes

,porte,limite_inferior_kg,limite_superior_kg,definicao
0,pequeno,0.0,0.5,"≤ 0,5 kg"
1,médio,0.6,3.0,"0,5 – 3 kg"
2,grande,3.1,20.0,3 – 20 kg
3,mega,20.1,Sem limite,> 20 kg


In [ ]:
df_classes.to_csv('classes_porte_animais_kg.csv', index=False)

In [ ]:
# Cria o documento
doc = Document()
doc.add_heading('Classes de porte por massa corporal (kg)', level=1)

doc.add_paragraph(
    'Classes definidas a partir da massa corporal em quilogramas: '
    'pequeno (≤ 0,5 kg), médio (0,5 – 3 kg), grande (3 – 20 kg), mega (> 20 kg).'
)

# Cria tabela: 1 linha de cabeçalho + N linhas de dados
n_rows = df_classes.shape[0] + 1
n_cols = df_classes.shape[1]

table = doc.add_table(rows=n_rows, cols=n_cols)

# Cabeçalho
hdr_cells = table.rows[0].cells
for j, col_name in enumerate(df_classes.columns):
    hdr_cells[j].text = str(col_name)

# Linhas de dados
for i, (_, row) in enumerate(df_classes.iterrows(), start=1):
    row_cells = table.rows[i].cells
    for j, col_name in enumerate(df_classes.columns):
        value = row[col_name]
        # Converte NaN para string vazia
        if isinstance(value, float) and pd.isna(value):
            value = ""
        row_cells[j].text = str(value)

# Salva o documento
output_filename = 'classes_porte_animais_kg.docx'
doc.save(output_filename)
print(f"Documento salvo como: {output_filename}")

Documento salvo como: classes_porte_animais_kg.docx


## Classificação das taxas de Pinto et al. (2022) por porte

In [ ]:
# -----------------------------------------------------------
# Classificação das taxas de Pinto et al. (2022) por porte
# -----------------------------------------------------------
# Este bloco utiliza a Tabela S2 (material suplementar) do artigo:
# Pinto F.A.S. et al. 2022. How Many Mammals Are Killed on Brazilian Roads?
# O arquivo CSV 'pinto_2022_table_s2.csv' foi construído a partir da tabela
# e contém as colunas em formato snake_case.

pinto_path = 'pinto_2022_table_s2.csv'

pinto_raw = pd.read_csv(pinto_path)

# Conferir colunas esperadas
expected_cols = [
    'scientific_name',
    'roadkill_records',
    'avg_est_ind_year_brazil',
    'median_est_ind_year_brazil',
    'max_est_ind_year_brazil',
    'min_est_ind_year_brazil',
    'avg_total_biomass_kg_year',
    'max_total_biomass_kg_year',
]
missing = [c for c in expected_cols if c not in pinto_raw.columns]
if missing:
    raise ValueError(f'Colunas ausentes no CSV de Pinto: {missing}')

# -----------------------------------------------------------
# Derivar massa corporal média (kg) a partir de biomassa/indivíduos
# -----------------------------------------------------------
# A tabela S2 apresenta a biomassa total média (kg/ano) e o número médio
# estimado de indivíduos atropelados por ano no Brasil.
# Uma estimativa consistente de massa por indivíduo é:
#   mass_kg = avg_total_biomass_kg_year / avg_est_ind_year_brazil

pinto_raw['mass_kg'] = (
    pinto_raw['avg_total_biomass_kg_year'] / pinto_raw['avg_est_ind_year_brazil']
)

# Remover linhas com valores não numéricos ou nulos de massa/taxa
pinto_clean = pinto_raw.dropna(subset=['mass_kg', 'avg_est_ind_year_brazil']).copy()

# -----------------------------------------------------------
# Função de classificação por porte (kg)
# -----------------------------------------------------------
def classify_body_mass_kg(mass_kg: float) -> str:
    if pd.isna(mass_kg):
        return "indefinido"
    if mass_kg <= 0.5:
        return "pequeno"
    if mass_kg <= 3.0:
        return "médio"
    if mass_kg <= 20.0:
        return "grande"
    return "mega"

# Aplicar a classificação
pinto_clean['porte'] = pinto_clean['mass_kg'].apply(classify_body_mass_kg)

# DataFrame detalhado por espécie, com taxa média (ind./ano) classificada por porte
df_taxas_pinto_porte = pinto_clean[[
    'scientific_name',
    'mass_kg',
    'porte',
    'avg_est_ind_year_brazil'
]].copy()

# -----------------------------------------------------------
# Resumo das taxas por classe de porte
# -----------------------------------------------------------
resumo_taxas_pinto_porte = (
    df_taxas_pinto_porte
    .groupby('porte')['avg_est_ind_year_brazil']
    .agg(['count', 'mean', 'min', 'max', 'sum'])
    .reset_index()
    .sort_values('porte')
)

df_taxas_pinto_porte

In [ ]:
# Exportar tabelas de taxas do Pinto classificadas por porte
df_taxas_pinto_porte.to_csv('taxas_pinto_por_porte_detalhado.csv', index=False)
resumo_taxas_pinto_porte.to_csv('taxas_pinto_por_porte_resumo.csv', index=False)